In [1]:
import os.path

import sqlalchemy as sql
import pandas as pd
import tensorflow as tf
import numpy as np

In [6]:
database = 'futurefactory'
ip = '145.38.197.14'
port = 5432
user = 'ffhackathon'
password = 'ff2022'

In [7]:
engine = sql.create_engine(f'postgresql://{user}:{password}@{ip}/{database}')
connection = engine.connect()

In [25]:
query = 'select a.fid, b.identificatie, ST_AsGeoJSON(a.geometrie, 15, 0) from brondata.bag3d_lod22_3d a left join brondata.bag3d_pand b on a.fid = b.fid where b.identificatie in (select identificatie from projectdata.eengezinswoningen) limit 100'

In [26]:
data = pd.read_sql(query, connection)


In [27]:
data.sample(5)

,fid,identificatie,st_asgeojson
10,12630123,0599100000058666,"{""type"":""MultiPolygon"",""coordinates"":[[[[97527..."
84,14074953,0599100000016236,"{""type"":""MultiPolygon"",""coordinates"":[[[[97975..."
52,12105695,0599100000109576,"{""type"":""MultiPolygon"",""coordinates"":[[[[97554..."
93,14078716,0599100000026676,"{""type"":""MultiPolygon"",""coordinates"":[[[[97864..."
72,8180561,0599100000260653,"{""type"":""MultiPolygon"",""coordinates"":[[[[97473..."


'{"type":"MultiPolygon","coordinates":[[[[97756.51798583985,442331.9954516601,-5.743000030517578],[97754.32090332032,442330.51132080075,-5.743000030517578],[97754.19370605469,442330.55581542966,-5.743000030517578],[97753.61973144532,442330.7566210937,-5.743000030517578],[97753.43912841797,442330.819822998,-5.743000030517578],[97753.30997802735,442331.0100085449,-5.743000030517578],[97753.17405273438,442330.91256591794,-5.743000030517578],[97751.11997558594,442329.4400012207,-5.743000030517578],[97753.6844897461,442325.64318725583,-5.743000030517578],[97754.60703613282,442324.27725097653,-5.743000030517578],[97754.72001220704,442324.1100146484,-5.743000030517578],[97750.06998779297,442320.93000671384,-5.743000030517578],[97750.03501464844,442320.9806658935,-5.743000030517578],[97749.69651367188,442321.47073242185,-5.743000030517578],[97748.18375732422,442323.66113159177,-5.743000030517578],[97748.15000488282,442323.70999023435,-5.743000030517578],[97748.38730957032,442323.85687133786,-5

In [41]:
import json
import numpy as np
np.array(json.loads(data.iloc[0]['st_asgeojson'])['coordinates']).shape


<ipython-input-41-1e67962d1bc8>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(json.loads(data.iloc[0]['st_asgeojson'])['coordinates']).shape


(77, 1)

In [44]:
json.loads(data.iloc[0]['st_asgeojson'])['coordinates'][0][0][0]

[97756.51798583985, 442331.9954516601, -5.743000030517578]

In [60]:
for i in range(10):
    print(len(json.loads(data.iloc[i]['st_asgeojson'])['coordinates']), len(json.loads(data.iloc[i]['st_asgeojson'])['coordinates'][0]), len(json.loads(data.iloc[i]['st_asgeojson'])['coordinates'][0][0]), len(json.loads(data.iloc[i]['st_asgeojson'])['coordinates'][0][0][0]))

77 1 54 3
61 1 39 3
43 1 34 3
23 1 20 3
68 1 44 3
24 1 21 3
30 1 27 3
44 1 33 3
22 1 17 3
31 1 23 3


In [62]:
x = json.loads(data.iloc[i]['st_asgeojson'])['coordinates']

In [64]:
fill = np.zeros(shape=(31, 1, 23, 3))

In [112]:
x = []
y = []
z = []

for i in json.loads(data.iloc[1]['st_asgeojson'])['coordinates']:
  for ii in i:
    for iii in ii:
        x.append(iii[0])
        y.append(iii[1])
        z.append(iii[2])

In [116]:
max_x = max(x)
max_y = max(y)
max_z = max(z)

x = [i/max_x for i in x]
y = [i/max_y for i in y]
z = [i/max_z for i in z]

In [117]:
df = pd.DataFrame()
df['x'] = x
df['y'] = y
df['z'] = z

In [118]:
import plotly.express as px
fig = px.scatter_3d(df, x='x', y='y', z='z')
fig.show()